In [39]:
import re
import pandas as pd
from datetime import datetime
import numpy as np

In [40]:
# create paths to each df to be parsed and aggregated
comment_path = "../Data/cleaned_wsb_comments.csv"
post_path = "../Data/cleaned_wsb_posts.csv"
ticker_path = "../Data/final_ticker_list.csv"

In [41]:
# create dfs for each source
%time comment_df = pd.read_csv(comment_path)
%time post_df = pd.read_csv(post_path)
%time ticker_df = pd.read_csv(ticker_path)

CPU times: user 27.3 s, sys: 6.2 s, total: 33.5 s
Wall time: 34.7 s
CPU times: user 3.99 s, sys: 487 ms, total: 4.47 s
Wall time: 4.56 s
CPU times: user 2.79 ms, sys: 547 µs, total: 3.34 ms
Wall time: 8.84 ms


In [42]:
# create copies of the large dfs so we dont have to read them in multiple times while testing
post_title_count_df = post_df.copy() # to count and save tickers in the post titles
post_selftext_count_df = post_df.copy() # to count and save tickers in the post selftext (body)
comment_body_count_df = comment_df.copy() # to count and save tickers in the comment body

In [43]:
# convert the utc date to standard us format (left it labeled utc, oh well)
comment_df["created_utc"] = pd.to_datetime(comment_df["created_utc"], unit="s")
post_df["created_utc"] = pd.to_datetime(post_df["created_utc"], unit="s")
post_title_count_df["created_utc"] = pd.to_datetime(post_df["created_utc"], unit="s")
post_selftext_count_df["created_utc"] = pd.to_datetime(post_df["created_utc"], unit="s")
comment_body_count_df["created_utc"] = pd.to_datetime(comment_df["created_utc"], unit="s")

In [44]:
%%time
# loop over all the tickers in our ticker list, and parse each target string in each df and append new row as ticker, mention
# str to upper bc all our tickers are upper

for ticker in ticker_df["Tickers"]:
    post_title_count_df[ticker] = np.where(post_title_count_df['title'].str.upper().str.contains(rf'\s({ticker})\s', na=False), 1, 0)
    post_selftext_count_df[ticker] = np.where(post_selftext_count_df['selftext'].str.upper().str.contains(rf'\s({ticker})\s', na=False), 1, 0)
    comment_body_count_df[ticker] = np.where(comment_body_count_df['body'].str.upper().str.contains(rf'\s({ticker})\s', na=False), 1, 0)

CPU times: user 29min 6s, sys: 8min 3s, total: 37min 10s
Wall time: 42min 34s


In [45]:
# change the date again for agg purposes
post_title_count_df["date"] = post_title_count_df["created_utc"].dt.date 
post_selftext_count_df["date"] = post_selftext_count_df["created_utc"].dt.date
comment_body_count_df["date"] = comment_body_count_df["created_utc"].dt.date

In [46]:
# check all the dfs
# post_title_count_df.head()
# post_selftext_count_df.head()
# comment_body_count_df.head()

In [47]:
# group dfs by date, and sum all mentions
comment_body_count_df_agg = comment_body_count_df.groupby('date')[ticker_df["Tickers"]].sum()
post_title_count_df_agg = post_title_count_df.groupby('date')[ticker_df["Tickers"]].sum()
post_selftext_count_agg = post_selftext_count_df.groupby('date')[ticker_df["Tickers"]].sum()

In [58]:
post_title_count_df_agg

,AMC,PUBM,GOGO,ISUN,CLVS,GME,SKT,LGND,TRIT,ESPR,...,JP,AAPL,ET,MSM,FB,TLRY,BRO,CRSR,IBKR,BIO
date,,,,,,,,,,,,,,,,,,,,,
2020-12-08,0,0,0,0,0,28,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0
2020-12-09,1,0,1,0,0,45,0,0,0,0,...,0,2,0,0,0,0,0,1,0,0
2020-12-10,0,0,0,0,0,11,0,0,0,0,...,1,0,0,0,4,0,0,2,0,0
2020-12-11,0,0,0,0,0,13,0,0,0,0,...,2,0,0,0,0,0,1,0,0,1
2020-12-12,1,0,0,0,0,11,0,0,0,0,...,0,0,0,0,0,0,1,0,2,0
2020-12-13,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-14,1,0,0,0,0,12,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2020-12-15,3,0,0,0,0,12,0,0,0,0,...,0,3,0,0,0,3,0,0,0,0
2020-12-16,0,0,0,0,0,10,0,0,1,0,...,1,1,0,0,2,6,0,1,1,0


In [59]:
post_selftext_count_agg

,AMC,PUBM,GOGO,ISUN,CLVS,GME,SKT,LGND,TRIT,ESPR,...,JP,AAPL,ET,MSM,FB,TLRY,BRO,CRSR,IBKR,BIO
date,,,,,,,,,,,,,,,,,,,,,
2020-12-08,0,0,0,0,0,12,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
2020-12-09,0,0,0,0,0,37,0,0,0,0,...,2,3,0,0,7,0,0,2,0,0
2020-12-10,3,0,0,0,0,6,0,0,0,0,...,2,2,1,0,5,0,1,0,1,0
2020-12-11,0,0,0,0,0,12,0,0,0,0,...,1,0,1,0,1,0,0,1,0,0
2020-12-12,0,0,0,0,0,5,0,0,0,0,...,3,3,0,0,2,0,0,2,2,0
2020-12-13,1,0,0,0,0,10,0,0,0,0,...,0,3,0,0,0,0,1,0,1,0
2020-12-14,1,0,0,0,0,11,0,0,0,0,...,4,4,2,0,0,0,0,2,0,0
2020-12-15,0,0,0,0,0,7,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2020-12-16,0,0,0,0,0,6,0,0,0,0,...,0,4,0,0,0,2,0,1,0,0


In [52]:
# save un aggregated df, might be useful later, not sure yet
post_title_count_df.to_csv("../Data/post_title_count_df.csv")
post_selftext_count_df.to_csv("../Data/post_selftext_count_df.csv")
comment_body_count_df.to_csv("../Data/comment_body_count_df.csv")

In [57]:
# save all agg by day dfs
comment_body_count_df_agg.to_csv("../Data/comment_body_count_agg.csv")
post_title_count_df_agg.to_csv("../Data/post_title_count_agg.csv")
post_selftext_count_agg.to_csv("../Data/post_selftext_count_agg.csv")